In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import XLNetTokenizer, XLNetForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
import torch
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.utils.class_weight import compute_class_weight

In [2]:
# Clear CUDA cache
torch.cuda.empty_cache()

# Run garbage collection
import gc
gc.collect()

# Verify the GPU is cleared
torch.cuda.empty_cache()

In [3]:
# Load the dataset
df = pd.read_csv("combined_output.csv", on_bad_lines='skip')

In [4]:
df.head()

,original_id,source_name,title,url,body,date_published,language,date_modified,author_list,images,description,sentiment,emotions,entities,quotations,prValues,clipping,label,category
0,'5404c9d2fd24852afa122f2cc01cb3acba3c5d05b682d...,'kabargayo.com,'Kredit sepeda motor bisa terbayar jika Anda m...,'https://www.kabargayo.com/2024/09/19/kredit-s...,"'Jakarta, VIVA – Pembayaran kredit sepeda moto...",19/09/2024 22.32,'id,19/09/2024 22.32,'Aldi Hadad,'https://i1.wp.com/thumb.viva.co.id/media/fron...,NaN,'positive,NaN,"'Hari Pembayaran Berbayar atau Harcilnas 2024,...",NaN,5250000,Adira,NaN,NaN
1,'e1e3f8d68b58568e8217b7562d48de634fceb0d837135...,'viva.co.id,'Kredit Motor Bisa Lunas Jika Bayar Cicilan Te...,'https://www.viva.co.id/otomotif/tips/1753596-...,"'Jakarta, VIVA – Cicilan kredit motor yang ser...",19/09/2024 22.30,'id,19/09/2024 22.30,'Krisna Wicaksono,'https://thumb.viva.co.id/media/frontend/thumb...,NaN,'positive,NaN,"'Harinya Cicilan Lunas,2024,PT Adira Dinamika ...",NaN,5250000,Adira,NaN,NaN
2,'dca74b8fa4eabf60cebfa7b811ecb385872a0fd301eaf...,'kabarmegapolitan.pikiran-rakyat.com,'Adira Finance Umumkan Pemenang HARCILNAS 2024...,'https://kabarmegapolitan.pikiran-rakyat.com/b...,'KABARMEGAPOLITAN.com - PT Adira Dinamika Mult...,19/09/2024 21.45,'id,19/09/2024 21.45,'Yuliansyah,'https://assets.pikiran-rakyat.com/www/network...,NaN,'positive,'HARCILNAS merupakan wujud apresiasi kami kepa...,"'PT Adira Dinamika,Cicilan Lunas HARCILNAS,12 ...","(Person :Tania Endah Budhi ,Quote : HARCILNAS ...",5250000,Adira,NaN,NaN
3,'56c73e2a6d254a17a5cc21dee7ed0b4660c3af70c093b...,'banggairaya.id,"'Dapatkan Promo Menarik, Yamaha Prima Motor Ra...",'https://banggairaya.id/dapatkan-promo-menarik...,'BANGGAI RAYA- Yamaha Prima Motor ramaikan pam...,19/09/2024 19.45,'id,19/09/2024 19.45,'Chikal Connect,'https://i0.wp.com/banggairaya.id/wp-content/u...,NaN,'neutral,NaN,"'RAYA- Yamaha Prima Motor,Banggai Goverment Ex...",NaN,5250000,Adira,NaN,NaN
4,'1cd6c6db60224b6ee5f49cd5d6c62cd2850d9f3255721...,'jakarta.tribunnews.com,"'Sindikat Penipuan Leasing, Satu Bulan Ajukan ...",'https://jakarta.tribunnews.com/2024/09/19/sin...,'Laporan wartawan TribunJakarta.com Yusuf Bach...,19/09/2024 18.43,'id,19/09/2024 18.43,'Yusuf Bachtiar,'https://asset-2.tstatic.net/jakarta/foto/bank...,NaN,'neutral,'Pelaku ini melakukan pembiayaan pembelian ken...,"'Yusuf Bachtiar TRIBUNJAKARTACOM,MEDAN,SATRIA,...","(Person :Dedi ,Quote : Pelaku ini melakukan pe...",5250000,Adira,NaN,NaN


In [5]:
# Clean extra characters from all columns in the DataFrame
for column in df.columns:
    if df[column].dtype == 'object':  # Check if the column is of string type
        df[column] = df[column].str.strip("'")  # Remove extra characters

# Filter relevant columns
df_filtered = df[['body', 'sentiment']].dropna()


In [6]:
df.head()

,original_id,source_name,title,url,body,date_published,language,date_modified,author_list,images,description,sentiment,emotions,entities,quotations,prValues,clipping,label,category
0,5404c9d2fd24852afa122f2cc01cb3acba3c5d05b682d4...,kabargayo.com,Kredit sepeda motor bisa terbayar jika Anda me...,https://www.kabargayo.com/2024/09/19/kredit-se...,"Jakarta, VIVA – Pembayaran kredit sepeda motor...",19/09/2024 22.32,id,19/09/2024 22.32,Aldi Hadad,https://i1.wp.com/thumb.viva.co.id/media/front...,NaN,positive,NaN,"Hari Pembayaran Berbayar atau Harcilnas 2024,P...",NaN,5250000,Adira,NaN,NaN
1,e1e3f8d68b58568e8217b7562d48de634fceb0d8371356...,viva.co.id,Kredit Motor Bisa Lunas Jika Bayar Cicilan Tep...,https://www.viva.co.id/otomotif/tips/1753596-k...,"Jakarta, VIVA – Cicilan kredit motor yang seri...",19/09/2024 22.30,id,19/09/2024 22.30,Krisna Wicaksono,https://thumb.viva.co.id/media/frontend/thumbs...,NaN,positive,NaN,"Harinya Cicilan Lunas,2024,PT Adira Dinamika M...",NaN,5250000,Adira,NaN,NaN
2,dca74b8fa4eabf60cebfa7b811ecb385872a0fd301eaf5...,kabarmegapolitan.pikiran-rakyat.com,Adira Finance Umumkan Pemenang HARCILNAS 2024:...,https://kabarmegapolitan.pikiran-rakyat.com/bi...,KABARMEGAPOLITAN.com - PT Adira Dinamika Multi...,19/09/2024 21.45,id,19/09/2024 21.45,Yuliansyah,https://assets.pikiran-rakyat.com/www/network/...,NaN,positive,HARCILNAS merupakan wujud apresiasi kami kepad...,"PT Adira Dinamika,Cicilan Lunas HARCILNAS,12 p...","(Person :Tania Endah Budhi ,Quote : HARCILNAS ...",5250000,Adira,NaN,NaN
3,56c73e2a6d254a17a5cc21dee7ed0b4660c3af70c093b2...,banggairaya.id,"Dapatkan Promo Menarik, Yamaha Prima Motor Ram...",https://banggairaya.id/dapatkan-promo-menarik-...,BANGGAI RAYA- Yamaha Prima Motor ramaikan pame...,19/09/2024 19.45,id,19/09/2024 19.45,Chikal Connect,https://i0.wp.com/banggairaya.id/wp-content/up...,NaN,neutral,NaN,"RAYA- Yamaha Prima Motor,Banggai Goverment Exp...",NaN,5250000,Adira,NaN,NaN
4,1cd6c6db60224b6ee5f49cd5d6c62cd2850d9f3255721d...,jakarta.tribunnews.com,"Sindikat Penipuan Leasing, Satu Bulan Ajukan K...",https://jakarta.tribunnews.com/2024/09/19/sind...,Laporan wartawan TribunJakarta.com Yusuf Bacht...,19/09/2024 18.43,id,19/09/2024 18.43,Yusuf Bachtiar,https://asset-2.tstatic.net/jakarta/foto/bank/...,NaN,neutral,Pelaku ini melakukan pembiayaan pembelian kend...,"Yusuf Bachtiar TRIBUNJAKARTACOM,MEDAN,SATRIA,T...","(Person :Dedi ,Quote : Pelaku ini melakukan pe...",5250000,Adira,NaN,NaN


In [7]:
# Balance dataset classes
class_counts = df_filtered['sentiment'].value_counts()
min_class = class_counts.min()
df_balanced = df_filtered.groupby('sentiment').apply(lambda x: x.sample(min_class)).reset_index(drop=True)

In [8]:
# Ensure balanced classes during split
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df_filtered['body'].tolist(), df_filtered['sentiment'].tolist(),
    test_size=0.2, random_state=42, stratify=df_filtered['sentiment']
)

In [9]:

# Load tokenizer and tokenize texts
tokenizer = XLNetTokenizer.from_pretrained("xlnet-base-cased")
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=512)


In [10]:
# Map sentiment labels to integers
label_mapping = {'positive': 2, 'neutral': 1, 'negative': 0}
train_labels = [label_mapping[label] for label in train_labels]
val_labels = [label_mapping[label] for label in val_labels]

In [11]:
class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


In [12]:
# Create datasets
train_dataset = SentimentDataset(train_encodings, train_labels)
val_dataset = SentimentDataset(val_encodings, val_labels)

In [13]:
# Check if GPU is available
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cuda


In [14]:
# Load XLNet model for sequence classification
model = XLNetForSequenceClassification.from_pretrained("xlnet-base-cased", num_labels=3)

Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'logits_proj.weight', 'sequence_summary.summary.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
class_weights = compute_class_weight('balanced', classes=np.unique(train_labels), y=train_labels)
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)

# Adjust model's loss function to include class weights
model.config.problem_type = "single_label_classification"
model.config.class_weights = class_weights.tolist()

In [16]:
from sklearn.metrics import accuracy_score
import numpy as np

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, preds)
    return {'eval_accuracy': acc}


In [17]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="steps",  # Evaluate more frequently
    eval_steps=200,              # Adjust based on dataset size
    save_strategy="steps",
    save_steps=200,
    per_device_train_batch_size=8,  # Lower batch size for better generalization
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=2,  # Simulates a larger batch size
    num_train_epochs=6,            # Train for more epochs
    learning_rate=1e-5,            # Lower learning rate for better fine-tuning
    weight_decay=0.01,
    warmup_steps=500,              # Gradual learning rate increase
    logging_dir="./logs",
    logging_steps=50,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="eval_accuracy",
    greater_is_better=True,
    lr_scheduler_type="cosine_with_restarts",  # More effective for fine-tuning
    fp16=True,                   # Enable mixed precision training
)


c:\Users\rache\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [18]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,  # Include custom metrics
)


In [19]:
import torch

torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats()

In [20]:
# Train the model
trainer.train()

  0%|          | 0/2700 [00:00<?, ?it/s]

{'loss': 1.1042, 'grad_norm': 23.382308959960938, 'learning_rate': 9.200000000000001e-07, 'epoch': 0.11}
{'loss': 1.0486, 'grad_norm': 32.777462005615234, 'learning_rate': 1.9000000000000002e-06, 'epoch': 0.22}
{'loss': 1.0547, 'grad_norm': 19.98434829711914, 'learning_rate': 2.9e-06, 'epoch': 0.33}
{'loss': 1.0455, 'grad_norm': 20.777698516845703, 'learning_rate': 3.900000000000001e-06, 'epoch': 0.44}


  0%|          | 0/113 [00:00<?, ?it/s]

{'eval_accuracy': 0.5008328706274292, 'eval_loss': 1.0246728658676147, 'eval_runtime': 32.9545, 'eval_samples_per_second': 54.651, 'eval_steps_per_second': 3.429, 'epoch': 0.44}
{'loss': 1.0223, 'grad_norm': 15.300000190734863, 'learning_rate': 4.9000000000000005e-06, 'epoch': 0.56}
{'loss': 1.0248, 'grad_norm': 18.869728088378906, 'learning_rate': 5.9e-06, 'epoch': 0.67}
{'loss': 1.0459, 'grad_norm': 19.955629348754883, 'learning_rate': 6.9e-06, 'epoch': 0.78}
{'loss': 1.0, 'grad_norm': 41.35318374633789, 'learning_rate': 7.9e-06, 'epoch': 0.89}


  0%|          | 0/113 [00:00<?, ?it/s]

...

  0%|          | 0/113 [00:00<?, ?it/s]

{'eval_accuracy': 0.7729039422543031, 'eval_loss': 0.6425918340682983, 'eval_runtime': 32.3772, 'eval_samples_per_second': 55.626, 'eval_steps_per_second': 3.49, 'epoch': 5.33}
{'loss': 0.3254, 'grad_norm': 16.377967834472656, 'learning_rate': 3.2784652327718544e-07, 'epoch': 5.44}
{'loss': 0.3603, 'grad_norm': 10.130207061767578, 'learning_rate': 2.1271516055901774e-07, 'epoch': 5.56}
{'loss': 0.335, 'grad_norm': 19.347028732299805, 'learning_rate': 1.2197870403878375e-07, 'epoch': 5.67}
{'loss': 0.3595, 'grad_norm': 29.1643009185791, 'learning_rate': 5.6099525900252805e-08, 'epoch': 5.78}


  0%|          | 0/113 [00:00<?, ?it/s]

{'eval_accuracy': 0.7712382009994447, 'eval_loss': 0.6537501811981201, 'eval_runtime': 32.3606, 'eval_samples_per_second': 55.654, 'eval_steps_per_second': 3.492, 'epoch': 5.78}
{'loss': 0.3643, 'grad_norm': 10.634465217590332, 'learning_rate': 1.541333133436018e-08, 'epoch': 5.89}
{'loss': 0.3826, 'grad_norm': 15.367070198059082, 'learning_rate': 1.2744786250407092e-10, 'epoch': 6.0}
{'train_runtime': 2740.192, 'train_samples_per_second': 15.765, 'train_steps_per_second': 0.985, 'train_loss': 0.6107909446292453, 'epoch': 6.0}


TrainOutput(global_step=2700, training_loss=0.6107909446292453, metrics={'train_runtime': 2740.192, 'train_samples_per_second': 15.765, 'train_steps_per_second': 0.985, 'total_flos': 1.2306930130944e+16, 'train_loss': 0.6107909446292453, 'epoch': 6.0})

In [21]:
print(f"Number of training examples: {len(train_dataset)}")
print(f"Batch size: {training_args.per_device_train_batch_size}")


Number of training examples: 7200
Batch size: 8


In [22]:
# Save the trained model and tokenizer
model.save_pretrained('./xlnet_model3')
tokenizer.save_pretrained('./xlnet_model3')

('./xlnet_model3\\tokenizer_config.json',
 './xlnet_model3\\special_tokens_map.json',
 './xlnet_model3\\spiece.model',
 './xlnet_model3\\added_tokens.json')

In [23]:
# Evaluate the model
results = trainer.evaluate()

  0%|          | 0/113 [00:00<?, ?it/s]

In [24]:
print(results)

{'eval_accuracy': 0.7773459189339256, 'eval_loss': 0.6148065328598022, 'eval_runtime': 32.4119, 'eval_samples_per_second': 55.566, 'eval_steps_per_second': 3.486, 'epoch': 6.0}


In [25]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report

# After training, get predictions on the validation dataset
predictions = trainer.predict(val_dataset)

# Get the predicted logits
logits = predictions.predictions

# Convert logits to predicted class labels
predicted_labels = np.argmax(logits, axis=1)

# Get the true labels
true_labels = val_labels

# Calculate evaluation metrics
accuracy = accuracy_score(true_labels, predicted_labels)
f1 = f1_score(true_labels, predicted_labels, average='weighted')
precision = precision_score(true_labels, predicted_labels, average='weighted')
recall = recall_score(true_labels, predicted_labels, average='weighted')

# Display metrics
print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")

# Detailed classification report
print("\nClassification Report:")
print(classification_report(true_labels, predicted_labels))


  0%|          | 0/113 [00:00<?, ?it/s]

Accuracy: 0.7773
F1 Score: 0.7757
Precision: 0.7781
Recall: 0.7773

Classification Report:
              precision    recall  f1-score   support

           0       0.79      0.63      0.70       338
           1       0.79      0.81      0.80       836
           2       0.76      0.81      0.78       627

    accuracy                           0.78      1801
   macro avg       0.78      0.75      0.76      1801
weighted avg       0.78      0.78      0.78      1801



In [26]:
# Check if GPU is available
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cuda
